In [1]:
from dziban.mkiv import Chart
from vega_datasets import data
from copy import deepcopy
import json
from tqdm import tqdm
import pandas as pd

In [2]:
movies = data('movies')
movies.head(1)

,Creative_Type,Director,Distributor,IMDB_Rating,IMDB_Votes,MPAA_Rating,Major_Genre,Production_Budget,Release_Date,Rotten_Tomatoes_Rating,Running_Time_min,Source,Title,US_DVD_Sales,US_Gross,Worldwide_Gross
0,None,None,Gramercy,6.1,1071.0,R,None,8000000.0,Jun 12 1998,NaN,NaN,None,The Land Girls,NaN,146083.0,146083.0


In [3]:
base = Chart(movies)

In [4]:
q1 = 'IMDB_Rating'
q2 = 'Rotten_Tomatoes_Rating'
q3 = 'Worldwide_Gross'
n1 = 'Major_Genre'
n2 = 'MPAA_Rating'
n3 = 'Creative_Type'

fields = {
    'q': [q1, q2, q3],
    'n': [n1, n2, n3]
}

marks = ['point', 'bar', 'line', 'area', 'tick', 'rect']
aggs = ['mean']

In [5]:
def get_props_from_transform(transform):
    props = {}
    if (transform == 'bin'):
        props['bin'] = True
    elif (transform == 'agg'):
        props['aggregate'] = 'mean'
        
    return props

In [6]:
def generate_queries(currdims, targetdims, curr_query, queries):
    if (currdims < targetdims):
        for fieldtype in ['q', 'n']:
            if (fieldtype == 'q'):
                for transform in ['raw', 'bin', 'agg']:
                    next_query = deepcopy(curr_query)
                    next_query.append({ 'fieldtype': fieldtype, 'transform': transform })
                    queries.append(next_query)

                    generate_queries(currdims + 1, targetdims, next_query, queries)
            else:
                next_query = deepcopy(curr_query)
                next_query.append({ 'fieldtype': fieldtype, 'transform': 'raw' })
                queries.append(next_query)

                generate_queries(currdims + 1, targetdims, next_query, queries)

In [7]:
def dedupe_queries(queries):
    seen = set()
    
    unique = []
    for query in queries:
        reps = {}
        for field in query:
            string = '{0}({1})'.format(field['transform'], field['fieldtype'])
            if string not in reps:
                reps[string] = 0
            reps[string] += 1
            
        stringified = json.dumps(reps, sort_keys=True)
        
        if stringified not in seen:
            seen.add(stringified)
            unique.append(query)
            
    return unique
            

In [8]:
def query_to_dziban(prior, query, available_fields):
    chart = prior
    
    used_fields = {}
    
    for field in query:
        fieldtype = field['fieldtype']
        transform = field['transform']
        
        fieldname = available_fields[fieldtype].pop(0)
    
        if (fieldtype not in used_fields):
            used_fields[fieldtype] = []
        used_fields[fieldtype].append(fieldname)
        
        props = get_props_from_transform(transform)
        
        props['ftype'] = 'quantitative' if fieldtype == 'q' else 'nominal'
            
        chart = chart.field(fieldname, **props)
        
    return {
        'chart': chart,
        'query': query,
        'available_fields': available_fields,
        'used_fields': used_fields
    }
        

In [9]:
priors = []
generate_queries(0, 2, [], priors)
priors = dedupe_queries(priors)

In [10]:
charts = [query_to_dziban(base, query, deepcopy(fields)) for query in priors]

In [11]:
mark_edits = []
for mark in marks:
    mark_edits.append({
        'type': 'mark',
        'mark': mark
    })

In [12]:
add_field_edits = []
for fieldtype in ['q', 'n']:
    if (fieldtype == 'q'):
        for transform in ['raw', 'bin', 'agg']:
            add_field_edits.append({
                'type': 'add_field',
                'fieldtype': fieldtype,
                'transform': transform
            })
            
    else:
        add_field_edits.append({
            'type': 'add_field',
            'fieldtype': fieldtype,
            'transform': 'raw'
        })

In [13]:
bin_edits = [
    {
        'type': 'bin'
    }
]

In [14]:
agg_edits = []
for agg in aggs:
    agg_edits.append(
        {
            'type': 'agg',
            'agg': agg
        }
    )

In [15]:
edits = mark_edits + add_field_edits + bin_edits + agg_edits

In [16]:
def edit_dziban(dzi, edit):
    edited_chart = dzi['chart']
    etype = edit['type']
    
    if etype == 'mark':
        edited_chart = edited_chart.mark(edit['mark'])
    elif etype == 'add_field':
        available_fields = dzi['available_fields']
        field = available_fields[edit['fieldtype']].pop(0)
        
        if (edit['fieldtype'] not in dzi['used_fields']):
            dzi['used_fields'][edit['fieldtype']] = []
        dzi['used_fields'][edit['fieldtype']].append(field)
        
        props = get_props_from_transform(edit['transform'])
        
        edited_chart = edited_chart.field(field, **props)
    elif etype == 'bin':
        used_fields = dzi['used_fields']
        
        if ('q' not in used_fields):
            return None
        
        field_to_bin = used_fields['q'][0]
        
        edited_chart = edited_chart.field(field_to_bin, bin=True)
    elif etype == 'agg':
        used_fields = dzi['used_fields']
        
        if ('q' not in used_fields):
            return None
        
        field_to_agg = used_fields['q'][0]
        edited_chart = edited_chart.field(field_to_agg, aggregate=edit['agg'])
        
    cold = edited_chart
    anchored = edited_chart.anchor_on(dzi['chart'])
    
    return {
        'prior': dzi['chart'],
        'prior_query': dzi['query'],
        'edit': edit,
        'cold': cold,
        'anchored': anchored,
        'available_fields': dzi['available_fields'],
        'used_fields': dzi['used_fields']
    }

In [17]:
nexts = []
for i in tqdm(range(len(charts))):
    chart = charts[i]
    for edit in edits:
        prior = deepcopy(chart)
        edited = edit_dziban(prior, edit)
        if edited is not None:
            nexts.append(edited)

100%|██████████| 14/14 [00:12<00:00,  1.20it/s]


In [18]:
with_differences = []
for i in tqdm(range(len(nexts))):
    n = nexts[i]
    prior = n['prior']
    cold = n['cold']
    anchored = n['anchored']
    
    if not cold.is_satisfiable():
        continue
        
    if not anchored.is_satisfiable():
        print(prior._get_vegalite())
        print(n['edit'])
        print('\n'.join(anchored._get_full_query()))
        print(cold._get_asp_complete())
        break
        
    cold_graphscape = set(cold - prior)
    anchored_graphscape = set(anchored._get_graphscape_list())
    
#     left_diff = cold_graphscape - anchored_graphscape
#     right_diff = anchored_graphscape - cold_graphscape
    
#     n['left_diff'] = left_diff
#     n['right_diff'] = right_diff
    
    n['cold_draco_rank'] = cold._get_draco_rank()
    n['cold_graphscape_rank'] = cold._get_graphscape_rank(prior)
    n['cold_draco_score'] = cold._get_draco_score()
    n['cold_graphscape_score'] = cold._get_graphscape_score(prior)
    
    n['anchored_draco_rank'] = anchored._get_draco_rank()
    n['anchored_graphscape_rank'] = anchored._get_graphscape_rank()
    n['anchored_draco_score'] = anchored._get_draco_score()
    n['anchored_graphscape_score'] = anchored._get_graphscape_score()

    

100%|██████████| 164/164 [41:30<00:00, 17.09s/it]


In [19]:
def stringify_query(query):
    result = ''
    for i, field in enumerate(query):
        if (i > 0):
            result += ' x '
    
        fieldtype = field['fieldtype']
        transform = field['transform']
        
        if fieldtype == 'n':
            result += 'n'
        elif fieldtype == 'q':
            if transform == 'raw':
                result += 'q'
            else:  
                result += '{0}({1})'.format(field['transform'], field['fieldtype'])
        
    return result

In [20]:
def stringify_edit(prior, edit):
    etype = edit['type']
    
    if etype == 'mark':
        return '{0} <> {1}'.format(prior._get_vegalite()['mark'], edit['mark'])
    elif etype == 'add_field':
        return '+ {0}({1})'.format(edit['transform'], edit['fieldtype'])
    elif etype == 'bin':
        return '<- bin(q)'
    elif etype == 'agg':
        return '<- agg(q)'

In [124]:
data = []
for i in tqdm(range(len(nexts))):
    n = nexts[i]
    prior_query = stringify_query(n['prior_query'])
    edit = stringify_edit(n['prior'], n['edit'])
    
#     left_diff = None
#     if ('left_diff' in n):
#         left_diff = json.dumps([x[0][5:] for x in n['left_diff']])
#         right_diff = json.dumps([x[0][5:] for x in n['right_diff']])

    if 'cold_draco_rank' not in n:
        data.append({
            'prior_query': prior_query,
            'edit': edit
        })
    else:
        cdr = n['cold_draco_rank']
        cgr = n['cold_graphscape_rank']
        adr = n['anchored_draco_rank']
        agr = n['anchored_graphscape_rank']
        
        cds = n['cold_draco_score']
        cgs = n['cold_graphscape_score']
        ads = n['anchored_draco_score']
        ags = n['anchored_graphscape_score']
        
        delta_dr = None
        delta_dr_assume = None
        if (cdr['rank'] is not None and adr['rank'] is not None):
            delta_dr = cdr['rank'] - adr['rank']
        
        if (cdr['rank'] is not None or adr['rank'] is not None):
            cdr_assume = cdr['rank'] if cdr['rank'] is not None else (cdr['of'] + 1)
            adr_assume = adr['rank'] if adr['rank'] is not None else (adr['of'] + 1)
            
            delta_dr_assume = cdr_assume - adr_assume
            
            
        delta_gr = None
        delta_gr_assume = None
        if (cgr['rank'] is not None and agr['rank'] is not None):
            delta_gr = cgr['rank'] - agr['rank']
        
        if (cgr['rank'] is not None or agr['rank'] is not None):
            cgr_assume = cgr['rank'] if cgr['rank'] is not None else (cgr['of'] + 1)
            agr_assume = agr['rank'] if agr['rank'] is not None else (agr['of'] + 1)
            
            delta_gr_assume = cgr_assume - agr_assume
        
        delta_sum_rank = None
        if (delta_dr is not None and delta_gr is not None):
            delta_sum_rank = delta_dr + delta_gr
            
        delta_sum_rank_assume = None
        if (delta_dr_assume is not None and delta_gr_assume is not None):
            delta_sum_rank_assume = delta_dr_assume + delta_gr_assume
        
            
        delta_ds = None
        if (cds is not None and ads is not None):
            delta_ds = cds - ads
    
        delta_gs = None
        if (cgs is not None and ags is not None):
            delta_gs = cgs - ags
        
        data.append({
            'prior_query': prior_query,
            'prior_dimensions': len(n['prior_query']),
            'edit_type': n['edit']['type'],
            'edit': edit,
#             'left_diff': left_diff,
#             'right_diff': right_diff,
            'cdr': cdr['rank'],
            'cgr': cgr['rank'],
            'adr': adr['rank'],
            'agr': agr['rank'],
            'delta_dr': delta_dr,
            'delta_gr': delta_gr,
            'anchored_gain_in_sum_rank': delta_sum_rank,
            'delta_dr_assume': delta_dr_assume,
            'delta_gr_assume': delta_gr_assume,
            'anchored_gain_in_sum_rank_assume': delta_sum_rank_assume,
            'cds': cds,
            'cgs': cgs,
            'ads': ads,
            'ags': ags
        })

100%|██████████| 164/164 [00:06<00:00, 26.31it/s]


In [313]:
columns = ['prior_query', 'prior_dimensions', 'edit', 'edit_type',
           'cdr', 'adr', 'cgr', 'agr',
           'delta_dr', 'delta_gr', 'anchored_gain_in_sum_rank',
           'delta_dr_assume', 'delta_gr_assume', 'anchored_gain_in_sum_rank_assume',
           'cds', 'cgs', 'ads', 'ags'
          ]

df = pd.DataFrame(data, columns=columns)
df.to_json('data.json', orient='records')
df.head(5)

,prior_query,prior_dimensions,edit,edit_type,cdr,adr,cgr,agr,delta_dr,delta_gr,anchored_gain_in_sum_rank,delta_dr_assume,delta_gr_assume,anchored_gain_in_sum_rank_assume,cds,cgs,ads,ags
0,q,1.0,tick <> point,mark,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,1.0,22.0,1.0
1,q,1.0,tick <> bar,mark,1.0,0.0,5.0,0.0,1.0,5.0,6.0,1.0,5.0,6.0,13.0,582.0,19.0,65.0
2,q,1.0,tick <> line,mark,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,583.0,39.0,583.0
3,q,1.0,tick <> area,mark,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,584.0,39.0,584.0
4,q,1.0,tick <> tick,mark,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,12.0,0.0


# Results

In [314]:
RESULTS = []
import math
import numpy as np

In [317]:
df = df[df['prior_dimensions'].notnull()]
df

,prior_query,prior_dimensions,edit,edit_type,cdr,adr,cgr,agr,delta_dr,delta_gr,anchored_gain_in_sum_rank,delta_dr_assume,delta_gr_assume,anchored_gain_in_sum_rank_assume,cds,cgs,ads,ags
0,q,1.0,tick <> point,mark,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,1.0,22.0,1.0
1,q,1.0,tick <> bar,mark,1.0,0.0,5.0,0.0,1.0,5.0,6.0,1.0,5.0,6.0,13.0,582.0,19.0,65.0
2,q,1.0,tick <> line,mark,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,583.0,39.0,583.0
3,q,1.0,tick <> area,mark,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,584.0,39.0,584.0
4,q,1.0,tick <> tick,mark,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,12.0,0.0
6,q,1.0,+ raw(q),add_field,1.0,0.0,NaN,9.0,1.0,NaN,NaN,1.0,32.0,33.0,12.0,1375.0,18.0,460.0
7,q,1.0,+ bin(q),add_field,0.0,2.0,NaN,5.0,-2.0,NaN,NaN,-2.0,36.0,34.0,15.0,1374.0,22.0,459.0
8,q,1.0,+ agg(q),add_field,0.0,0.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,583.0,26.0,583.0
9,q,1.0,+ raw(n),add_field,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,459.0,30.0,459.0
10,q,1.0,<- bin(q),bin,0.0,2.0,13.0,6.0,-2.0,7.0,5.0,-2.0,7.0,5.0,13.0,582.0,20.0,522.0


In [318]:
def add_label(records, key, value):
    for r in records:
        r[key] = value
        
    return records

In [319]:
def summarize(dataframe):
    result = []
    
    def measures(frame):
        result = []
        
        dr_mean = frame['delta_dr'].mean()
        dr_std = frame['delta_dr'].std()
        
        gr_mean = frame['delta_gr'].mean()
        gr_std = frame['delta_gr'].std()
        
        tr_mean = frame['anchored_gain_in_sum_rank'].mean()
        tr_std = frame['anchored_gain_in_sum_rank'].std()
        
        print('MEAN')
        print('Gain in Draco rank (stdev):                            {0} ({1})'.format(dr_mean, dr_std))
        print('Gain in Graphscape rank:                               {0} ({1})'.format(gr_mean, gr_std))
        print('Gain in Total rank (both above must be defined):       {0} ({1})'.format(tr_mean, tr_std))
        print()
        
        def h(val):
            return val if not np.isnan(val) else None
            
        result.append({
            'dr_mean': h(dr_mean),
            'dr_std': h(dr_std),
            'gr_mean': h(gr_mean),
            'gr_std': h(gr_std),
            'tr_mean': h(tr_mean),
            'tr_std': h(tr_std)
        })
              
        return result

    print('%% ALL %%')
    result += add_label(measures(dataframe), 'include_zero_deltas', True)
    
#     print('%% EXCLUDING ZERO GAIN %%')
#     frame = dataframe[(dataframe['delta_dr'] != 0) & (dataframe['delta_gr'] != 0)]
#     result += add_label(measures(frame), 'include_zero_deltas', False)
    
    return result



In [320]:
def summarize_group(f):
    result = []
    
    print('%%%%% TOTAL %%%%%%')
    result += add_label(summarize(f), 'edit_type', 'total')
    print('%%%%%%%%%%%%%%%%%\n\n')
    
    print('%%%%% MARK %%%%%%')
    mark = f[f['edit_type'] == 'mark']
    result += add_label(summarize(mark), 'edit_type', 'mark')
    print('%%%%%%%%%%%%%%%%%\n\n')
    
    print('%%%%% ADD FIELD %%%%%%')
    add_field = f[f['edit_type'] == 'add_field']
    result += add_label(summarize(add_field), 'edit_type', 'add_field')
    print('%%%%%%%%%%%%%%%%%\n\n')
    
    print('%%%%% AGGREGATE %%%%%%')
    add_field = f[f['edit_type'] == 'agg']
    result += add_label(summarize(add_field), 'edit_type', 'agg')
    print('%%%%%%%%%%%%%%%%%\n\n')
    
    print('%%%%% BIN %%%%%%')
    add_field = f[f['edit_type'] == 'bin']
    result += add_label(summarize(add_field), 'edit_type', 'bin')
    print('%%%%%%%%%%%%%%%%%\n\n')
    
    return result


## Aggregate

In [321]:
RESULTS += add_label(summarize_group(df), 'dims', 'all')

%%%%% TOTAL %%%%%%
%% ALL %%
MEAN
Gain in Draco rank (stdev):                            -2.7246376811594204 (7.788038850560508)
Gain in Graphscape rank:                               3.280701754385965 (7.376725863073329)
Gain in Total rank (both above must be defined):       1.4513274336283186 (7.010185383785814)

%%%%%%%%%%%%%%%%%


%%%%% MARK %%%%%%
%% ALL %%
MEAN
Gain in Draco rank (stdev):                            -1.6 (6.131443108648615)
Gain in Graphscape rank:                               1.7702702702702702 (3.844751973037081)
Gain in Total rank (both above must be defined):       0.16216216216216217 (5.805115899228904)

%%%%%%%%%%%%%%%%%


%%%%% ADD FIELD %%%%%%
%% ALL %%
MEAN
Gain in Draco rank (stdev):                            -5.3478260869565215 (10.562443722515368)
Gain in Graphscape rank:                               8.333333333333334 (12.82547284209001)
Gain in Total rank (both above must be defined):       5.0 (9.959006887326577)

%%%%%%%%%%%%%%%%%


%%%%% AGGREGA

## 1D

In [322]:
oned = df[df['prior_dimensions'] == 1.0]

In [323]:
RESULTS += add_label(summarize_group(oned), 'dims', 1)

%%%%% TOTAL %%%%%%
%% ALL %%
MEAN
Gain in Draco rank (stdev):                            -1.9743589743589745 (6.815181116978475)
Gain in Graphscape rank:                               3.235294117647059 (9.078976125457705)
Gain in Total rank (both above must be defined):       1.1470588235294117 (6.16997791333003)

%%%%%%%%%%%%%%%%%


%%%%% MARK %%%%%%
%% ALL %%
MEAN
Gain in Draco rank (stdev):                            -0.85 (4.042862456491101)
Gain in Graphscape rank:                               0.45 (1.3945382182304158)
Gain in Total rank (both above must be defined):       -0.4 (3.4701736646971852)

%%%%%%%%%%%%%%%%%


%%%%% ADD FIELD %%%%%%
%% ALL %%
MEAN
Gain in Draco rank (stdev):                            -3.8666666666666667 (9.869626325729044)
Gain in Graphscape rank:                               9.4 (15.28397853963424)
Gain in Total rank (both above must be defined):       4.2 (9.818350166906862)

%%%%%%%%%%%%%%%%%


%%%%% AGGREGATE %%%%%%
%% ALL %%
MEAN
Gain in Draco ran

## 2D

In [324]:
twod = df[df['prior_dimensions'] == 2.0]
RESULTS += add_label(summarize_group(twod), 'dims', 2)

%%%%% TOTAL %%%%%%
%% ALL %%
MEAN
Gain in Draco rank (stdev):                            -3.0202020202020203 (8.152850757187664)
Gain in Graphscape rank:                               3.3 (6.588059778536441)
Gain in Total rank (both above must be defined):       1.5822784810126582 (7.375648767530006)

%%%%%%%%%%%%%%%%%


%%%%% MARK %%%%%%
%% ALL %%
MEAN
Gain in Draco rank (stdev):                            -1.8727272727272728 (6.744145734153562)
Gain in Graphscape rank:                               2.259259259259259 (4.331478396728736)
Gain in Total rank (both above must be defined):       0.37037037037037035 (6.475778663157031)

%%%%%%%%%%%%%%%%%


%%%%% ADD FIELD %%%%%%
%% ALL %%
MEAN
Gain in Draco rank (stdev):                            -6.064516129032258 (10.966419907672597)
Gain in Graphscape rank:                               7.571428571428571 (11.304963414456404)
Gain in Total rank (both above must be defined):       5.615384615384615 (10.42064026774476)

%%%%%%%%%%%%%%%%%



In [325]:
without_identicals = df[(df['delta_dr'] != 0) | (df['delta_gr'] != 0)]

In [326]:
len(without_identicals)

73

In [327]:
len(df)

149

In [328]:
summarize_group(without_identicals)

%%%%% TOTAL %%%%%%
%% ALL %%
MEAN
Gain in Draco rank (stdev):                            -6.064516129032258 (10.753376850852801)
Gain in Graphscape rank:                               9.842105263157896 (9.993312130618735)
Gain in Total rank (both above must be defined):       4.4324324324324325 (11.802872862477498)

%%%%%%%%%%%%%%%%%


%%%%% MARK %%%%%%
%% ALL %%
MEAN
Gain in Draco rank (stdev):                            -5.0 (10.159596020725816)
Gain in Graphscape rank:                               5.695652173913044 (5.067138179294428)
Gain in Total rank (both above must be defined):       0.5217391304347826 (10.565249922687762)

%%%%%%%%%%%%%%%%%


%%%%% ADD FIELD %%%%%%
%% ALL %%
MEAN
Gain in Draco rank (stdev):                            -7.6875 (11.974266763193855)
Gain in Graphscape rank:                               20.0 (12.692955176439847)
Gain in Total rank (both above must be defined):       12.777777777777779 (12.686388155990043)

%%%%%%%%%%%%%%%%%


%%%%% AGGREGATE %%%%

[{'dr_mean': -6.064516129032258,
  'dr_std': 10.753376850852801,
  'gr_mean': 9.842105263157896,
  'gr_std': 9.993312130618735,
  'tr_mean': 4.4324324324324325,
  'tr_std': 11.802872862477498,
  'include_zero_deltas': True,
  'edit_type': 'total'},
 {'dr_mean': -5.0,
  'dr_std': 10.159596020725816,
  'gr_mean': 5.695652173913044,
  'gr_std': 5.067138179294428,
  'tr_mean': 0.5217391304347826,
  'tr_std': 10.565249922687762,
  'include_zero_deltas': True,
  'edit_type': 'mark'},
 {'dr_mean': -7.6875,
  'dr_std': 11.974266763193855,
  'gr_mean': 20.0,
  'gr_std': 12.692955176439847,
  'tr_mean': 12.777777777777779,
  'tr_std': 12.686388155990043,
  'include_zero_deltas': True,
  'edit_type': 'add_field'},
 {'dr_mean': -1.0,
  'dr_std': 0.0,
  'gr_mean': 15.0,
  'gr_std': 9.899494936611665,
  'tr_mean': 14.0,
  'tr_std': 9.899494936611665,
  'include_zero_deltas': True,
  'edit_type': 'agg'},
 {'dr_mean': -2.0,
  'dr_std': 1.4142135623730951,
  'gr_mean': 4.333333333333333,
  'gr_std': 2.